In [2]:
import requests
from bs4 import BeautifulSoup as bs
import pymongo
import re

In [3]:
conn = pymongo.MongoClient()
cine_db = conn.cine21
actors = cine_db.actors

In [4]:
cine21_url = "http://www.cine21.com/rank/person/content"
header = {}
header['section'] = 'actor'
header['period_start'] = '2020-03'
header['gender'] = 'all'
header['page'] = 1

res = requests.post(cine21_url, data=header)

In [5]:
soup = bs(res.content, 'html.parser')
print(soup)

 <ul class="people_list">
<li class="people_li">
<a href="/db/person/info/?person_id=2552"><img alt="" class="people_thumb" src="http://image.cine21.com/resize/cine21/person/2019/1211/11_32_33__5df055413c01a[X145,145].jpg" target="_blank"/></a>
<div class="name"><a href="/db/person/info/?person_id=2552">전도연(1편)</a></div>
<ul class="num_info">
<li><span class="tit">흥행지수</span><strong>71,991</strong></li>
<!--
						<li><a href="#" class="btn_graph"><span class="ico"></span><span>흥행성적<br />그래프로 보기</span></a></li>
						-->
</ul>
<!-- 영화포스터는 최대 5개까지만 -->
<ul class="mov_list">
<li>
<a href="/movie/info/?movie_id=54143">
<img alt="" class="thumb" src="http://image.cine21.com/resize/cine21/poster/2020/0212/16_42_16__5e43ac5844ca0[X85,120].jpg" target="_blank"/>
<span>지푸라기라도 잡고 싶은 짐승들</span>
</a>
</li>
</ul>
<!-- 순위 --><span class="grade">1</span>
</li>
<li class="people_li">
<a href="/db/person/info/?person_id=13093"><img alt="" class="people_thumb" src="http://image.cine21.com/resize/cine21

In [6]:
actors = soup.select("ul.people_list li.people_li div.name")

actor_list = []
for actor in actors:
    actor_name = re.sub('\(\w*\)','',actor.text)
    actor_link = actor.select_one('a').attrs['href']
    actor_links = requests.get("http://www.cine21.com/"+actor_link)
    actor_response = bs(actor_links.content, 'html.parser')
    default_info = actor_response.select_one("ul.default_info")
    actor_details = default_info.select("li")
    
    actor_dict = {}
    for actor_detail in actor_details:
        actor_schema = actor_detail.select_one('span.tit').text
        actor_value = re.sub("<span.*?>.*?</span>",'',str(actor_detail))
        actor_value = re.sub("<.*?>",'',actor_value).strip()
        actor_dict[actor_schema] = actor_value
    actor_list.append(actor_dict)
print(actor_list)

[{'다른 이름': 'Jeon Do Youn', '직업': '배우', '생년월일': '1973-02-11', '성별': '여', '신장/체중': '165cm, 45kg', '학교': '서울예술대학 방송연예', '취미': '음악듣기, 영화감상', '특기': '수상스키, 포켓볼', '소속사': '노아엔터테인먼트'}, {'원어명': '윤여정', '직업': '배우', '생년월일': '1947-06-19', '성별': '여', '학교': '한양대학교 국문학 (중퇴)'}, {'직업': '배우', '생년월일': '1973-03-20', '성별': '남', '홈페이지': 'https://www.instagram.com/tojws/\nhttps://www.facebook.com/officialjws', '신장/체중': '186cm, 79kg', '학교': '경기상업고등학교 중퇴', '취미': '비디오 감상，드라이브', '특기': '농구，수영', '소속사': '(주)토러스 필름'}, {'직업': '배우', '생년월일': '1972-11-21', '성별': '남', '신장/체중': '177cm, 72kg'}, {'직업': '배우', '생년월일': '1975-03-06', '성별': '여', '홈페이지': 'https://www.facebook.com/cjesmiran'}, {'직업': '배우', '생년월일': '1982-05-22', '성별': '남', '홈페이지': 'https://twitter.com/k_moo'}, {'직업': '배우', '성별': '남'}]


In [8]:
# 배우 이름과 흥행지수 따오기

actors = soup.select("ul.people_list li.people_li div.name")
hits = soup.select("ul.people_list li.people_li ul.num_info li strong")
movie = soup.select("ul.people_list")
for index, actor in enumerate(actors):
    actor_name = re.sub("\(\w*\)",'',actor.text)
    actor_hits = int(hits[index].text.replace(',',''))
    
    
    


AttributeError: ResultSet object has no attribute 'select_one'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?